In [1]:
import os 
import csv
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
random.seed(42)
rnd = [random.randrange(189), random.randrange(189), random.randrange(189)]
print(rnd)

[163, 28, 6]


In [3]:
reuse_distances = pd.DataFrame()
test_distances = pd.DataFrame()
tmp_ctr = 0
for file in os.listdir("../ce_data/reuse_distances"):
    x = pd.read_csv("../ce_data/reuse_distances/"+file)
    if tmp_ctr in rnd:
        test_distances = pd.concat([test_distances, x], ignore_index=True)
    else:
        reuse_distances = pd.concat([reuse_distances, x], ignore_index=True)
    tmp_ctr +=1

In [4]:
cache_misses = pd.read_csv("../ce_data/caches/llc_miss_ratio.csv")
test_misses = cache_misses.loc[cache_misses['id'].isin(test_distances['id'])]
cache_misses = cache_misses.loc[~cache_misses['id'].isin(test_distances['id'])]

In [5]:
cache_misses = cache_misses.sort_values("id", ascending=True, ignore_index=True)
reuse_distances = reuse_distances.sort_values("id", ascending=True, ignore_index=True)
test_misses = test_misses.sort_values("id", ascending=True, ignore_index=True)
test_distances = test_distances.sort_values("id", ascending=True, ignore_index=True)

In [6]:
td_ids = test_distances["id"]
test_distances = test_distances.drop(columns=["id"])
tm_ids = test_misses["id"]
test_misses = test_misses.drop(columns=["id",'32','64','128','256','512'])

In [7]:
print(cache_misses)
print(reuse_distances)
print(test_distances)
print(test_misses)

                           id        32        64       128       256  \
0         400.perlbench-41B_0  1.000000  1.000000  1.000000  0.998652   
1         400.perlbench-41B_1  1.000000  1.000000  1.000000  1.000000   
2        400.perlbench-41B_10  1.000000  0.997375  0.997375  0.997375   
3       400.perlbench-41B_100  0.996656  0.996656  0.996656  0.993311   
4       400.perlbench-41B_101  1.000000  1.000000  1.000000  1.000000   
...                       ...       ...       ...       ...       ...   
139309        657.xz_s-56B_95  1.000000  1.000000  1.000000  1.000000   
139310        657.xz_s-56B_96  1.000000  1.000000  1.000000  1.000000   
139311        657.xz_s-56B_97  1.000000  1.000000  1.000000  1.000000   
139312        657.xz_s-56B_98  1.000000  1.000000  1.000000  1.000000   
139313        657.xz_s-56B_99  1.000000  1.000000  1.000000  1.000000   

             512      1024      2048      4096      8192  
0       0.889488  0.828841  0.803235  0.773585  0.768194  
1    

In [8]:
# Quick check if all columns are the same
for i,j in zip(reuse_distances['id'], cache_misses['id']):
    if i!=j:
        print(i, j)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(reuse_distances,cache_misses, test_size=0.1)

In [10]:
X_train_ids = X_train["id"]
X_test_ids = X_test["id"]
y_train_ids = y_train["id"]
y_test_ids = y_test["id"]

print(X_train_ids)

123645    648.exchange2_s-387B_152
51780          464.h264ref-97B_188
52252          464.h264ref-97B_612
122270    648.exchange2_s-210B_263
119821    648.exchange2_s-1227B_82
                    ...           
132219        654.roms_s-1390B_454
1143         400.perlbench-50B_453
111248       641.leela_s-1083B_455
128063       649.fotonik3d_s-1B_85
101181          625.x264_s-12B_158
Name: id, Length: 125382, dtype: object


In [11]:
X_train = X_train.drop(columns=["id"])
X_test = X_test.drop(columns=["id"])
y_train = y_train.drop(columns=["id",'32','64','128','256','512'])
y_test = y_test.drop(columns=["id",'32','64','128','256','512'])

In [12]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(125382, 896) (13932, 896)
(125382, 4) (13932, 4)


In [13]:
print(X_train.columns)

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '3623878656', '3690987520', '3758096384', '3825205248', '3892314112',
       '3959422976', '4026531840', '4093640704', '4160749568', '4227858432'],
      dtype='object', length=896)


In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.metrics import MeanSquaredError
from keras.losses import MeanSquaredError
# Create the model
model = Sequential()

# Add the input layer and the first hidden layer
model.add(Dense(896, input_dim=896, activation='sigmoid'))

# Add the second hidden layer
model.add(Dense(896, activation='sigmoid'))

# Add the output layer
model.add(Dense(4, activation='sigmoid'))

# Compile the model
model.compile(loss=MeanSquaredError() , optimizer='adam')

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 896)               803712    
                                                                 
 dense_1 (Dense)             (None, 896)               803712    
                                                                 
 dense_2 (Dense)             (None, 4)                 3588      
                                                                 
Total params: 1,611,012
Trainable params: 1,611,012
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(X_train.to_numpy(), y_train.to_numpy(), epochs=10)

Epoch 1/10
3919/3919 [==============================] - 39s 10ms/step - loss: 0.0724
Epoch 2/10
3919/3919 [==============================] - 36s 9ms/step - loss: 0.0671
Epoch 3/10
3919/3919 [==============================] - 36s 9ms/step - loss: 0.0636
Epoch 4/10
3919/3919 [==============================] - 35s 9ms/step - loss: 0.0645
Epoch 5/10
3919/3919 [==============================] - 38s 10ms/step - loss: 0.0613
Epoch 6/10
3919/3919 [==============================] - 38s 10ms/step - loss: 0.0601
Epoch 7/10
3919/3919 [==============================] - 38s 10ms/step - loss: 0.0594
Epoch 8/10
3919/3919 [==============================] - 39s 10ms/step - loss: 0.0615
Epoch 9/10
3919/3919 [==============================] - 38s 10ms/step - loss: 0.0604
Epoch 10/10
3919/3919 [==============================] - 39s 10ms/step - loss: 0.0586


In [16]:
f = model(X_test.to_numpy())

In [17]:
from sklearn.metrics import mean_squared_error
# Compare columns
for i,j in enumerate(['1024','2048','4096','8192']):
    model_out = f.numpy()[:,i]
    y_out = y_test.to_numpy()[:,i]
    print(f'Mean squared error for cache size {j}\t: {mean_squared_error(model_out, y_out)}')

Mean squared error for cache size 1024	: 0.05517712466736137
Mean squared error for cache size 2048	: 0.061540295183689364
Mean squared error for cache size 4096	: 0.060909559515391544
Mean squared error for cache size 8192	: 0.05465849192179149


In [18]:
print(test_distances)

           0      1      2      3      4      5      6      7     8     9  \
0       5648   1600    752    656    608    336    320    208   320   288   
1       5120   1472    976    560    528    336    272    272   240   432   
2       9184   2048   1104   1040    800    496    400    544   384   416   
3      15024   3520   1584   1744   1200    672    752    784   704   608   
4      12976   3072   1440   1440    912    464    560    624   448   544   
...      ...    ...    ...    ...    ...    ...    ...    ...   ...   ...   
2242  184576  66640  35056  28272  20944  23552  16976  13216  8832  6448   
2243  186304  65232  35600  28096  19408  21680  17824  14496  8416  6368   
2244  184192  66160  33264  27136  20144  22800  16416  13152  7712  6256   
2245  186352  66448  35088  28672  20768  22592  17120  14608  8720  6448   
2246  188240  63200  32016  27152  18912  20800  15872  13216  7088  5520   

      ...  3623878656  3690987520  3758096384  3825205248  3892314112  \
0 

In [19]:
f = model(test_distances.to_numpy())

In [20]:
print(f)

tf.Tensor(
[[0.3471112  0.27875435 0.22926319 0.20406228]
 [0.2759319  0.20317009 0.18533567 0.18423703]
 [0.1692524  0.09300341 0.05433223 0.03941225]
 ...
 [0.44662878 0.38883492 0.32705888 0.23140106]
 [0.44662878 0.38883492 0.32705888 0.23140106]
 [0.4466288  0.38883483 0.32705888 0.23140106]], shape=(2247, 4), dtype=float32)


In [21]:
print(test_misses)

          1024      2048      4096      8192
0     0.046590  0.000969  0.000962  0.000962
1     0.047124  0.001620  0.001620  0.001620
2     0.049473  0.001464  0.001464  0.001464
3     0.049063  0.003882  0.003766  0.003766
4     0.051722  0.002382  0.002318  0.002318
...        ...       ...       ...       ...
2242  0.000000  0.000000  0.000000  0.000000
2243  0.000000  0.000000  0.000000  0.000000
2244  0.000000  0.000000  0.000000  0.000000
2245  0.000000  0.000000  0.000000  0.000000
2246  0.000000  0.000000  0.000000  0.000000

[2247 rows x 4 columns]


In [22]:
from sklearn.metrics import mean_squared_error
# Compare columns
for i,j in enumerate(['1024','2048','4096','8192']):
    model_out = f.numpy()[:,i]
    y_out = test_misses.to_numpy()[:,i]
    print(f'Mean squared error for cache size {j}\t: {mean_squared_error(model_out, y_out)}')

Mean squared error for cache size 1024	: 0.10148151760593702
Mean squared error for cache size 2048	: 0.10351027023400965
Mean squared error for cache size 4096	: 0.08842017717430908
Mean squared error for cache size 8192	: 0.07278161739104462
